來源: https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.ml.clustering.KMeans.html

In [1]:
# https://www.kaggle.com/arjunbhasin2013/ccdata


#api for k-means algo
from pyspark.ml.clustering import KMeans,KMeansModel

#ClusteringEvaluator
from pyspark.ml.evaluation import ClusteringEvaluator
#https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.ml.evaluation.ClusteringEvaluator.html
#The metric computes the Silhouette measure using the squared Euclidean distance.

from numpy import array
from math import sqrt

#Build the api for the entrance
from pyspark.context import SparkContext,SparkConf
from pyspark.rdd import RDD
from pyspark.sql import SparkSession
import os
import sys


#Config
os.environ['JAVA_HOME'] = 'C:\Program Files\Java\jre1.8.0_301'
os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable


sc = SparkContext(conf=SparkConf().setAppName("Kmeans").setMaster("local"))
print(f'sc:{sc}')
spark = SparkSession(sc)

sc:<SparkContext master=local appName=Kmeans>


## 使用df  ##是使用ml package


In [6]:
from pyspark.ml.linalg import Vectors
data = [(Vectors.dense([0.0, 0.0]), 2.0), (Vectors.dense([1.0, 1.0]), 2.0),
        (Vectors.dense([9.0, 8.0]), 2.0), (Vectors.dense([8.0, 9.0]), 2.0)]
df = spark.createDataFrame(data, ["features", "weighCol"])
df

DataFrame[features: vector, weighCol: double]

In [5]:
kmeans = KMeans(k=2) #2群
kmeans.setSeed(1)
kmeans.setWeightCol("weighCol")#????
kmeans.setMaxIter(10)#10 iterations
kmeans.getMaxIter() #若不呼叫只會回傳位置


10

In [8]:

kmeans.clear(kmeans.maxIter)#???
model = kmeans.fit(df)#訓練模型
#model.getDistanceMeasure()



In [9]:
model.getDistanceMeasure() #預設是'euclidean'
##'euclidean'

'euclidean'

In [10]:
model.setPredictionCol("newPrediction")
#用途未知

KMeansModel: uid=KMeans_a52ce599020d, k=2, distanceMeasure=euclidean, numFeatures=2

In [11]:
model.predict(df.head().features)

#?????

0

# 中心  mean

In [13]:
centers = model.clusterCenters()
centers  

[array([0.5, 0.5]), array([8.5, 8.5])]

In [15]:
len(centers) #取得2個重心

2

In [20]:
#?????
newPredictiontransformed = model.transform(df).select("features", "newPrediction")

newPredictiontransformed.collect()

[Row(features=DenseVector([0.0, 0.0]), newPrediction=0),
 Row(features=DenseVector([1.0, 1.0]), newPrediction=0),
 Row(features=DenseVector([9.0, 8.0]), newPrediction=1),
 Row(features=DenseVector([8.0, 9.0]), newPrediction=1)]

In [21]:
rows = transformed.collect()
rows

[Row(features=DenseVector([0.0, 0.0]), newPrediction=0),
 Row(features=DenseVector([1.0, 1.0]), newPrediction=0),
 Row(features=DenseVector([9.0, 8.0]), newPrediction=1),
 Row(features=DenseVector([8.0, 9.0]), newPrediction=1)]

## 驗證區

In [22]:
rows[0].newPrediction == rows[1].newPrediction

True

In [23]:
rows[2].newPrediction == rows[3].newPrediction

True

In [24]:
rows[0].newPrediction == rows[3].newPrediction

False

In [28]:
print(model.hasSummary)
summary = model.summary
summary.k

True


2

In [29]:
summary.clusterSizes


[2, 2]

In [30]:
summary.trainingCost


4.0

## 存檔區

In [37]:
temp_path = r"C:\\\Users\\\LeoShr\\\p_space\\\NTHU\MDA\\\CH3_K_means\\\dataset"
kmeans_path = temp_path + "\kmeans"
kmeans.save(kmeans_path)

Py4JJavaError: An error occurred while calling o524.save.
: java.io.IOException: Path C:\\\Users\\\LeoShr\\\p_space\\\NTHU\MDA\\\CH3_K_means\\\dataset\kmeans already exists. To overwrite it, please use write.overwrite().save(path) for Scala and use write().overwrite().save(path) for Java and Python.
	at org.apache.spark.ml.util.FileSystemOverwrite.handleOverwrite(ReadWrite.scala:683)
	at org.apache.spark.ml.util.MLWriter.save(ReadWrite.scala:167)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(Unknown Source)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(Unknown Source)
	at java.lang.reflect.Method.invoke(Unknown Source)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:357)
	at py4j.Gateway.invoke(Gateway.java:282)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(C

In [ ]:
#load in model data
kmeans2 = KMeans.load(kmeans_path)
kmeans2.getK()

In [ ]:
#load in model data

model_path = temp_path + "/kmeans_model"
model.save(model_path)

In [ ]:
model2 = KMeansModel.load(model_path)
model2.hasSummary

In [ ]:
model.clusterCenters()[0] == model2.clusterCenters()[0]
array([ True,  True], dtype=bool)
model.clusterCenters()[1] == model2.clusterCenters()[1]
array([ True,  True], dtype=bool)
model.transform(df).take(1) == model2.transform(df).take(1)